In [6]:
# 記事反映
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import urllib.request as req
import glob
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

from sqlalchemy import (Column, String, Text, ForeignKey, \
                create_engine, MetaData, DECIMAL, DATETIME, exc, event, Index)
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.orm import (sessionmaker, relationship, scoped_session)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import INTEGER as Integer
from datetime import datetime
from sqlalchemy import Column, Integer, create_engine, MetaData, DATETIME, String,Date,Float
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import datetime
from sqlalchemy import create_engine, text

print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()

def login_scraping_site(area_name):
    global browser
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)
    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))
    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))
    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser


browser = login_scraping_site('chubu')
#エリアの都道府県の名前と対応番号を取得
prefectures_box_elements = browser.find_element(By.CLASS_NAME,"one_sp_block")
prefectures_elements:list = prefectures_box_elements.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
prefecture_name_and_number_dict:dict[str:str] = {}

for element in prefectures_elements:
    prefecture_name:str = element.find_element(By.CLASS_NAME,"side_bar_L_O").text
    #print(prefecture_name)
    link_url:str = element.find_element(By.TAG_NAME,'a').get_attribute('href')
    prefecture_number:int = link_url.split('ken=')[-1]
    #print(prefecture_number)
    prefecture_name_and_number_dict[prefecture_name] = prefecture_number
prefecture_name_and_number_dict

読み込み完了
ライブラリの読み込み完了


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\2930225511.py:61: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()


{'愛知県': '23',
 '静岡県': '22',
 '山梨県': '19',
 '長野県': '20',
 '岐阜県': '21',
 '新潟県': '15',
 '富山県': '16',
 '石川県': '17',
 '福井県': '18',
 '三重県': '24'}

In [8]:
from  src.utils import *

In [7]:
i = 0
#取材取得日指定
target_date :datetime = datetime.date.today() + datetime.timedelta(days=1)
target_date_str_sql:str = target_date.strftime('%Y-%m-%d')
#合成用の空のDF
furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','取材ランク'])


for prefecture_name,prefecture_number in prefecture_name_and_number_dict.items():
    print(prefecture_name)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
    browser.get(url)
    browser.implicitly_wait(10)
    kiji_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
    kiji_elements_list:list = kiji_element_box.find_elements(By.CLASS_NAME,"osbox")
    for i,syuzai_record_element in enumerate(kiji_elements_list):
        if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
            browser = login_scraping_site('chubu')
            url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
            browser.get(url)
            browser.implicitly_wait(10)
        else:
            pass
        #print(syuzai_record_element.text)
        syuzai_record_element_list = syuzai_record_element.find_elements(By.CLASS_NAME,"list_event_name")
        browser.implicitly_wait(1)
        print(syuzai_record_element_list)
        if len(syuzai_record_element_list) == 0:
            continue
        tenpo_name = syuzai_record_element.find_element(By.CLASS_NAME,"oslh2").text
        for syuzai_name_element in syuzai_record_element_list:
            #print(syuzai_name_element.text)
            syuzai_rank = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_rank").text
            schedule_name = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_li").text
            print(tenpo_name,syuzai_rank,schedule_name)
            record = pd.Series([prefecture_name,target_date, tenpo_name,schedule_name,syuzai_rank], index=furture_syuzai_list_df.columns)
            furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
    break
#browser.quit()


愛知県
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="6b30e5b6-9e34-409c-a2f0-0df72ecbdf09")>]
キクヤ春日井店 S ナビ子AI予想【差枚プラス】★3 (3のつく日)
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="6a2667cf-6d38-4e1c-bc21-61558e4bb530")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="1bb2ba2a-2cca-4756-9dc2-4e445c75769c")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="8cd5e015-d660-4e57-b4b5-b1a7b7adb1f2")>]
キング会館 名東店 A ナビ子AI予想【差枚プラス】★1 (3のつく日)
キング会館 名東店 C ナビ子AI予想【機種仕掛け】★1 (3のつく日)
キング会館 名東店 C エース小吉来店
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="d97b17d7-8630-4882-b5b5-0af3be1e8bf4")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="7f38b2bd-4659-4281-94d8-45ba00a2a439")>]


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from

オーギヤ半田店 C ナビ子AI予想【機種仕掛け】★1 (3のつく日)
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="cc42522c-e3f5-463f-b5d8-1736ca5f0e7f")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="baa72240-5f35-4d45-9b28-9c5d7ebcdd62")>]
ニューサンプラザ S ナビ子AI予想【差枚プラス】★3 (3のつく日)
ニューサンプラザ B ナビ子AI予想【機種仕掛け】★2 (3のつく日)
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="894cf97d-d74b-47b0-bb24-ad39ef05d24c")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="5c9fa385-1d07-44c6-8564-6725abe6e2ee")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="df2b469b-d75e-49a4-b27b-262180e11c4a")>]
サンシャインKYORAKU 栄 C ナビ子AI予想【機種仕掛け】★1 (3のつく日)


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from

サンシャインKYORAKU 栄 ・ 旧イベ(3のつく日)
サンシャインKYORAKU 栄 ・ 新台入替(P-WORLD引用)
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="c35e0636-dd50-4fe7-96ac-9b413094fcf5")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="dc89d5eb-2cbb-47dd-b79b-3fa25b573177")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="a712f984-4334-4e96-a8c8-7c7a1be3eadf")>]
サンシャインKYORAKU南 C ナビ子AI予想【機種仕掛け】★1 (3のつく日)
サンシャインKYORAKU南 ・ スロッター×スロッター
サンシャインKYORAKU南 ・ 旧イベ(3のつく日)
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="6196cb4f-6cc7-4e8d-a9ee-0b28d84297bf")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="c2ff8c4b-c08f-408b-a223-25d2fb02aa8f")>]


...

[]
[]
[]
[]
[]
[]
[]
[]
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="48210d1d-80ce-4416-b4e2-b4a528e9756f")>]
コンコルド880愛西日比野駅前店 ・ 光るん×スロッター


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)


[]
[]
[<selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="a788cb4d-c195-4212-825e-a9bd16a1ee3b")>, <selenium.webdriver.remote.webelement.WebElement (session="8b4f0250f9114ece9d1f2038b9aeb844", element="12f35725-e1f2-4bdc-8998-cc48cf1ef470")>]
ミカド稲沢店 ・ 最光超
ミカド稲沢店 ・ ジャックランタンOVER


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)


[]


In [40]:

furture_syuzai_list_df

,都道府県,イベント日,店舗名,取材名,取材ランク
0,愛知県,2023-03-23,キクヤ春日井店,ナビ子AI予想【差枚プラス】★3 (3のつく日),S
1,愛知県,2023-03-23,キング会館 名東店,ナビ子AI予想【差枚プラス】★1 (3のつく日),A
2,愛知県,2023-03-23,キング会館 名東店,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
3,愛知県,2023-03-23,キング会館 名東店,エース小吉来店,C
4,愛知県,2023-03-23,オーギヤ半田店,ナビ子AI予想【差枚プラス】★1 (3のつく日),A
5,愛知県,2023-03-23,オーギヤ半田店,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
6,愛知県,2023-03-23,ニューサンプラザ,ナビ子AI予想【差枚プラス】★3 (3のつく日),S
7,愛知県,2023-03-23,ニューサンプラザ,ナビ子AI予想【機種仕掛け】★2 (3のつく日),B
8,愛知県,2023-03-23,サンシャインKYORAKU 栄,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
9,愛知県,2023-03-23,サンシャインKYORAKU 栄,旧イベ(3のつく日),・


In [9]:
twitter = UtilsTwitterClass()
print(vars(twitter))

username = os.getenv('DB_USER_NAME')
password=os.getenv('DB_PASSWORD')
host=os.getenv('DB_HOST')
port='3306'
database=os.getenv('DB_NAME')
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"

# DB接続するためのEngineインスタンス
engine = create_engine(db_url, echo=False)

# Sessionの作成
session = scoped_session(sessionmaker(autocommit = False,autoflush = False,bind = engine))
# modelで使用する
Base = declarative_base()
Base.query = session.query_property()



query = f"select * from schedule where 店舗名 = 'キクヤ春日井店' AND イベント日 = '{twitter.tomorrow_date_string_sql}'"
df = pd.read_sql_query(sql=text(query), con=engine.connect())
df.sort_values('イベント日')

{'tomorrow_8numbers_string': '20230323', 'tomorrow_date_stinrg_jp': '3/23(木)', 'tomorrow_date_string_sql': '2023-03-23', 'yesterday': datetime.date(2023, 3, 21), 'tomorrow': datetime.date(2023, 3, 23), 'yesterday_date_string': '3/21(火)', 'id': '', 'pw': '', 'tweet_text': '', 'image_path_list': [], 'main_tweet_text': '', 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site', 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image', 'tweet_footer_text': '\n\n#XXXX 20230321\n\n', 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image', 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json'}


C:\Users\tsc95\AppData\Local\Temp\ipykernel_45596\173008262.py:17: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,3368,愛知県,2023-03-23,木,キクヤ春日井店,ナビ子AI予想【差枚プラス】★1 (3のつく日),ホールナビ,A,2023-03-16 11:05:27
1,4545,愛知県,2023-03-23,木,キクヤ春日井店,ナビ子AI予想【差枚プラス】★1 (3のつく日),ホールナビ,A,2023-03-18 11:06:06
2,4992,愛知県,2023-03-23,木,キクヤ春日井店,ナビ子AI予想【差枚プラス】★3 (3のつく日),ホールナビ,S,2023-03-18 11:06:06


In [10]:
pledge_df = pd.read_csv(r'csv\取材公約用マスター.csv')
display(pledge_df)
convert_parlar_name_df = pd.read_csv(r'csv\店舗名変換テーブル.csv')
convert_parlar_name_df = convert_parlar_name_df.rename(columns={'ホールナビ店舗名':'店舗名'})
convert_parlar_name_df

,取材名,媒体名,公約内容
0,ガチアンケート金色,ガチアン!,ガチアン!が行うユーザーアンケート
1,ｙｔｒ来店実践,SEVENSTV,来店実践
2,Ａレポート,アツ姫,Aタイプに高設定投入
3,黒バラの極意(来店ver),ジャンバリ.TV,NaN
4,鮮魚レポート,アメスロ,NaN
...,...,...,...
753,AceOVER,アツ姫,NaN
754,APチャンネル来店収録,APチャンネル,NaN
755,APチャンネル来店,APチャンネル,NaN
756,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践


,都道府県,店舗kintone,shop_id,店舗名,DMM店舗名,アナスロ店舗名,緯度,経度,住所
0,北海道,P01378,1378,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,43.085828,141.514724,北海道江別市大麻泉町47-37
1,北海道,P01582,1582,ひまわり音更店,音更ひまわり,ひまわり音更店,42.961369,143.207331,北海道河東郡音更町木野大通東13丁目1番21
2,北海道,P01351,13160,EBH,EBH,EBH,43.106287,141.292820,北海道札幌市西区発寒15条4-13-1
3,茨城県,P02562,2562,キコーナ水戸駅南店,キコーナ水戸駅南店,キコーナ水戸駅南店,36.365522,140.477010,茨城県水戸市城南1-4-1
4,神奈川県,P04933,4933,ピーズ・タウン桜ヶ丘,P’Stown桜ヶ丘,ピーズ・タウン桜ヶ丘,35.451103,139.465162,神奈川県大和市福田1-6-10
...,...,...,...,...,...,...,...,...,...
2352,東京都,P00363,363,楽園渋谷駅前店,楽園渋谷駅前店,楽園渋谷駅前店,35.658285,139.699800,東京都渋谷区道玄坂1-3-1
2353,山梨県,P12714,13138,マルハン甲府店,マルハン甲府店,マルハン甲府店,35.631724,138.553588,山梨県甲府市国母8-20-10
2354,東京都,P12738,13170,メッセ西荻窪店,メッセ西荻窪店,メッセ西荻窪店,35.703498,139.600920,東京都杉並区西荻南3-14-7
2355,埼玉県,P12792,13227,マルキ戸田店,マルキ戸田店,マルキ戸田店,NaN,NaN,埼玉県戸田市美女木東1-3-2


In [11]:
parlar_name_list = list(furture_syuzai_list_df['店舗名'].unique())
print('parlar_name_list_len:',len(parlar_name_list))
# マーカープロット（ポップアップ設定，色変更，アイコン変更）
text = ''
tomorrow_schedule_df = pd.DataFrame(index=[], columns=[])
for parlar_name in parlar_name_list:
    query = f"select * from schedule WHERE 店舗名 = '{parlar_name}' AND イベント日 = '{target_date_str_sql}'"
    print(query)
    extract_parlar_name_df = pd.read_sql_query(sql=query, con=engine.connect())
    extract_parlar_name_df =  extract_parlar_name_df[~extract_parlar_name_df['取材名'].str.contains('ナビ子')]
    extract_parlar_name_df = extract_parlar_name_df[~extract_parlar_name_df.duplicated(subset=['イベント日','曜日','店舗名','取材名','媒体名'])]
    display(extract_parlar_name_df)
    tomorrow_schedule_df = pd.concat([tomorrow_schedule_df, extract_parlar_name_df])
    #display(convert_parlar_name_df)
    #tenpo_syuzai_merge_df = pd.merge(extract_parlar_name_df, convert_parlar_name_df, on='店舗名')
    # merge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
    # merged_df =merge_df[merge_df['都道府県_x'] == '愛知県'].sort_values('取材ランク')
tomorrow_schedule_df
# tenpo_syuzai_merge_df = pd.merge(tomorrow_schedule_df, convert_parlar_name_df, on='店舗名')
# #tenpo_syuzai_merge_df
# merge_df = pd.merge(tomorrow_schedule_df,pledge_df, on='取材名')
# merge_df

parlar_name_list_len: 24
select * from schedule WHERE 店舗名 = 'キクヤ春日井店' AND イベント日 = '2023-03-23'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間


select * from schedule WHERE 店舗名 = 'キング会館 名東店' AND イベント日 = '2023-03-23'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
1,3749,愛知県,2023-03-23,木,キング会館 名東店,エース小吉来店,その他取材・企画,C,2023-03-17 11:05:46


select * from schedule WHERE 店舗名 = 'オーギヤ半田店' AND イベント日 = '2023-03-23'


...

select * from schedule WHERE 店舗名 = 'コンコルド880愛西日比野駅前店' AND イベント日 = '2023-03-23'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,3315,愛知県,2023-03-23,木,コンコルド880愛西日比野駅前店,光るん×スロッター,パチンコ店長のホール攻略,・,2023-03-16 11:05:27


select * from schedule WHERE 店舗名 = 'ミカド稲沢店' AND イベント日 = '2023-03-23'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,3798,愛知県,2023-03-23,木,ミカド稲沢店,最光超,アツ姫,・,2023-03-17 11:05:46


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
1,3749,愛知県,2023-03-23,木,キング会館 名東店,エース小吉来店,その他取材・企画,C,2023-03-17 11:05:46
1,3441,愛知県,2023-03-23,木,サンシャインKYORAKU 栄,旧イベ(3のつく日),旧イベ,・,2023-03-16 11:05:27
0,3312,愛知県,2023-03-23,木,サンシャインKYORAKU南,スロッター×スロッター,パチンコ店長のホール攻略,・,2023-03-16 11:05:27
2,3440,愛知県,2023-03-23,木,サンシャインKYORAKU南,旧イベ(3のつく日),旧イベ,・,2023-03-16 11:05:27
0,3296,愛知県,2023-03-23,木,MEGAコンコルド1280稲沢店,その日、最も盛り上がった光景を取材する,パチンコウォーカー,・,2023-03-16 11:05:27
1,3444,愛知県,2023-03-23,木,グランドオータ888鳴海店,旧イベ(3のつく日),旧イベ,・,2023-03-16 11:05:27
0,3239,愛知県,2023-03-23,木,プレイランドキャッスル知多東海店,でちゃう編集部来店(かおりん),でちゃう！,C,2023-03-16 11:05:27
0,3228,愛知県,2023-03-23,木,メガガイア一宮,降臨シリーズ オーディン,DMMぱちタウン,C,2023-03-16 11:05:27
1,3445,愛知県,2023-03-23,木,メガガイア一宮,旧イベ(3のつく日),旧イベ,・,2023-03-16 11:05:27
0,3241,愛知県,2023-03-23,木,プレイランドキャッスル知多にしの台店,でちゃう編集部来店(かおりん),でちゃう！,C,2023-03-16 11:05:27


In [ ]:
html_table_df = merge_df[['イベント日','曜日','店舗名','取材名','媒体名_x','公約内容']]
html_table_df['イベント日'] = twitter.tomorrow_date_stinrg_jp
html_table_df = html_table_df[['イベント日','店舗名','取材名','媒体名_x','公約内容']]
print(html_table_df.to_html(index=False))

In [12]:
furture_syuzai_list_df

,都道府県,イベント日,店舗名,取材名,取材ランク
0,愛知県,2023-03-23,キクヤ春日井店,ナビ子AI予想【差枚プラス】★3 (3のつく日),S
1,愛知県,2023-03-23,キング会館 名東店,ナビ子AI予想【差枚プラス】★1 (3のつく日),A
2,愛知県,2023-03-23,キング会館 名東店,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
3,愛知県,2023-03-23,キング会館 名東店,エース小吉来店,C
4,愛知県,2023-03-23,オーギヤ半田店,ナビ子AI予想【差枚プラス】★1 (3のつく日),A
5,愛知県,2023-03-23,オーギヤ半田店,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
6,愛知県,2023-03-23,ニューサンプラザ,ナビ子AI予想【差枚プラス】★3 (3のつく日),S
7,愛知県,2023-03-23,ニューサンプラザ,ナビ子AI予想【機種仕掛け】★2 (3のつく日),B
8,愛知県,2023-03-23,サンシャインKYORAKU 栄,ナビ子AI予想【機種仕掛け】★1 (3のつく日),C
9,愛知県,2023-03-23,サンシャインKYORAKU 栄,旧イベ(3のつく日),・


In [13]:
merged_syuzai_pledge_df = pd.merge(furture_syuzai_list_df,pledge_df,how='left',on='取材名')
merged_syuzai_pledge_df = merged_syuzai_pledge_df[~merged_syuzai_pledge_df['取材名'].str.contains('ナビ子')]
merged_syuzai_pledge_df = merged_syuzai_pledge_df.fillna('未調査')
merged_syuzai_pledge_df

,都道府県,イベント日,店舗名,取材名,取材ランク,媒体名,公約内容
3,愛知県,2023-03-23,キング会館 名東店,エース小吉来店,C,その他取材・企画,未調査
9,愛知県,2023-03-23,サンシャインKYORAKU 栄,旧イベ(3のつく日),・,未調査,未調査
10,愛知県,2023-03-23,サンシャインKYORAKU 栄,新台入替(P-WORLD引用),・,新台入替,未調査
12,愛知県,2023-03-23,サンシャインKYORAKU南,スロッター×スロッター,・,パチンコ店長のホール攻略,4つの公約が存在。要HPチェック
13,愛知県,2023-03-23,サンシャインKYORAKU南,旧イベ(3のつく日),・,未調査,未調査
14,愛知県,2023-03-23,MEGAコンコルド1280稲沢店,最光取材,・,未調査,未調査
15,愛知県,2023-03-23,MEGAコンコルド1280稲沢店,その日、最も盛り上がった光景を取材する,・,パチンコウォーカー,未調査
17,愛知県,2023-03-23,グランドオータ888鳴海店,旧イベ(3のつく日),・,未調査,未調査
18,愛知県,2023-03-23,プレイランドキャッスル知多東海店,でちゃう編集部来店(かおりん),C,でちゃう！,未調査
19,愛知県,2023-03-23,プレイランドキャッスル知多東海店,打ちたい台が固まっているか聞かせてくれ,・,でちゃう！,未調査


In [14]:

def recommend_image(write_image_context, length,read_image_path,save_image_jpg):

    image = Image.open(read_image_path)
    draw = ImageDraw.Draw(image)
    # フォントを指定
    font_path = r"font\LightNovelPOPv2.otf"
    font = ImageFont.truetype(font_path, size=40)

    # 文字を描く
    # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
    # fillはRGBで文字の色を決めています
    draw.multiline_text((55, 40), write_image_context, fill=(255, 255, 255), font=font, spacing=10, stroke_width=5, stroke_fill=(55, 55, 55))

    # 画像を保存する
    image.save(save_image_jpg)
    print('length', length)
    length_croped = 110 + length * 66  # 500
    im = Image.open(save_image_jpg)
    im_crop = im.crop((0, 0, 1000, length_croped))
    im_crop.save(save_image_jpg, quality=100)

In [15]:
merged_syuzai_pledge_df

,都道府県,イベント日,店舗名,取材名,取材ランク,媒体名,公約内容
3,愛知県,2023-03-23,キング会館 名東店,エース小吉来店,C,その他取材・企画,未調査
9,愛知県,2023-03-23,サンシャインKYORAKU 栄,旧イベ(3のつく日),・,未調査,未調査
10,愛知県,2023-03-23,サンシャインKYORAKU 栄,新台入替(P-WORLD引用),・,新台入替,未調査
12,愛知県,2023-03-23,サンシャインKYORAKU南,スロッター×スロッター,・,パチンコ店長のホール攻略,4つの公約が存在。要HPチェック
13,愛知県,2023-03-23,サンシャインKYORAKU南,旧イベ(3のつく日),・,未調査,未調査
14,愛知県,2023-03-23,MEGAコンコルド1280稲沢店,最光取材,・,未調査,未調査
15,愛知県,2023-03-23,MEGAコンコルド1280稲沢店,その日、最も盛り上がった光景を取材する,・,パチンコウォーカー,未調査
17,愛知県,2023-03-23,グランドオータ888鳴海店,旧イベ(3のつく日),・,未調査,未調査
18,愛知県,2023-03-23,プレイランドキャッスル知多東海店,でちゃう編集部来店(かおりん),C,でちゃう！,未調査
19,愛知県,2023-03-23,プレイランドキャッスル知多東海店,打ちたい台が固まっているか聞かせてくれ,・,でちゃう！,未調査


In [29]:
read_image_path = r"image\image_source\eva_board.jpg"
prefecture_name = '愛知県'
create_image_number = 0
parlar_name_count = 0
schedule_text = ''
last_time_parlar_name = ''

for i,row in merged_syuzai_pledge_df.iterrows():
    #print(row)
    print(row['店舗名'],last_time_parlar_name )
    if last_time_parlar_name != row['店舗名']:
        schedule_text += '\n◆' +  row['店舗名'] + '\n'
        parlar_name_count += 1
        #print(parlar_name_count)
    else:
        pass


    if row['取材名'].startswith(tuple(['旧イベ'])):
        schedule_text += '　☆' + row['取材名']  + '\n'
        last_time_parlar_name = row['店舗名']
        continue
    else:
        schedule_text += '　☆' + row['取材名'] + ' 【' + row['媒体名'] + '】' + '\n'
        

    if str(row['公約内容']) == 'nan':
        schedule_text += '       ┗' + '未調査' + '\n'
    else:
        schedule_text += '       ┗' + row['公約内容'] + '\n'

    last_time_parlar_name = row['店舗名']
    
    #print(parlar_name_count)
    if (parlar_name_count > 9 ) or merged_syuzai_pledge_df.index[-1] == i :
        #print('処理開始')
        create_image_number  += 1
        title = f'{prefecture_name} {twitter.tomorrow_date_stinrg_jp} 明日のスロットイベントまとめ{create_image_number}\n'
        input_image_text : str = title + schedule_text
        text__row_length = input_image_text.count('\n')
        #print('length', text__row_length )
        print(input_image_text)
        save_image_jpg = fr"image\temp_image\eva_board_{prefecture_name}_{target_date_str_sql}_{create_image_number}.jpg"
        recommend_image(input_image_text, text__row_length ,read_image_path,save_image_jpg)
        schedule_text = ''
        last_time_parlar_name = ''
        parlar_name_count = 0
        
    
 
print(input_image_text)

キング会館 名東店 
サンシャインKYORAKU 栄 キング会館 名東店
サンシャインKYORAKU 栄 サンシャインKYORAKU 栄
サンシャインKYORAKU南 サンシャインKYORAKU 栄
サンシャインKYORAKU南 サンシャインKYORAKU南
MEGAコンコルド1280稲沢店 サンシャインKYORAKU南
MEGAコンコルド1280稲沢店 MEGAコンコルド1280稲沢店
グランドオータ888鳴海店 MEGAコンコルド1280稲沢店
プレイランドキャッスル知多東海店 グランドオータ888鳴海店
プレイランドキャッスル知多東海店 プレイランドキャッスル知多東海店
メガガイア一宮 プレイランドキャッスル知多東海店
メガガイア一宮 メガガイア一宮
プレイランドキャッスル知多にしの台店 メガガイア一宮
プレイランドキャッスル知多にしの台店 プレイランドキャッスル知多にしの台店
サンシャインKYORAKU平針 プレイランドキャッスル知多にしの台店
キング666会館半田店 サンシャインKYORAKU平針
愛知県 3/23(木) 明日のスロットイベントまとめ1

◆キング会館 名東店
　☆エース小吉来店 【その他取材・企画】
       ┗未調査

◆サンシャインKYORAKU 栄
　☆旧イベ(3のつく日)
　☆新台入替(P-WORLD引用) 【新台入替】
       ┗未調査

◆サンシャインKYORAKU南
　☆スロッター×スロッター 【パチンコ店長のホール攻略】
       ┗4つの公約が存在。要HPチェック
　☆旧イベ(3のつく日)

◆MEGAコンコルド1280稲沢店
　☆最光取材 【未調査】
       ┗未調査
　☆その日、最も盛り上がった光景を取材する 【パチンコウォーカー】
       ┗未調査

◆グランドオータ888鳴海店
　☆旧イベ(3のつく日)

◆プレイランドキャッスル知多東海店
　☆でちゃう編集部来店(かおりん) 【でちゃう！】
       ┗未調査
　☆打ちたい台が固まっているか聞かせてくれ 【でちゃう！】
       ┗未調査

◆メガガイア一宮
　☆降臨シリーズ オーディン 【DMMぱちタウン】
       ┗3台並び以上で4か所以上に高設定
　☆旧イベ(3のつく日)

◆プレイランドキャッスル知多にしの台店
　